In [1]:
import qutip as qu
import numpy as np
import jax.numpy as jnp
import jax


In [8]:
seed = 1
d = 2
example_process = qu.rand_super(2, seed=seed)
example_process

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 0.57905439+0.j         -0.01341539+0.05313417j -0.01341539-0.05313417j
   0.42094561+0.j        ]
 [ 0.02138688+0.05045602j  0.26332321-0.03787573j  0.01191271-0.00902282j
  -0.02138688-0.05045602j]
 [ 0.02138688-0.05045602j  0.01191271+0.00902282j  0.26332321+0.03787573j
  -0.02138688+0.05045602j]
 [ 0.42094561+0.j          0.01341539-0.05313417j  0.01341539+0.05313417j
   0.57905439+0.j        ]]

In [10]:
def wrap_L(L):
    return qu.Qobj(np.array(L), dims=[[[2], [2]], [[2], [2]]], shape=(4, 4), type="super")

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 0.57905439+0.j         -0.01341539+0.05313417j -0.01341539-0.05313417j
   0.42094561+0.j        ]
 [ 0.02138688+0.05045602j  0.26332321-0.03787573j  0.01191271-0.00902282j
  -0.02138688-0.05045602j]
 [ 0.02138688-0.05045602j  0.01191271+0.00902282j  0.26332321+0.03787573j
  -0.02138688+0.05045602j]
 [ 0.42094561+0.j          0.01341539-0.05313417j  0.01341539+0.05313417j
   0.57905439+0.j        ]]

In [12]:
N = 2  #dimension of Hilbert Space
G = np.array([qu.identity(N), qu.sigmax(), qu.sigmay(), qu.sigmaz()]) / np.sqrt(N)


In [342]:
np.transpose(example_process.full().reshape([2, 2, 2, 2]), [2, 0, 3, 1]).reshape([4, 4])

array([[ 0.57905439+0.j        ,  0.02138688+0.05045602j,
        -0.01341539+0.05313417j,  0.26332321-0.03787573j],
       [ 0.02138688-0.05045602j,  0.42094561+0.j        ,
         0.01191271+0.00902282j,  0.01341539-0.05313417j],
       [-0.01341539-0.05313417j,  0.01191271-0.00902282j,
         0.42094561+0.j        , -0.02138688-0.05045602j],
       [ 0.26332321+0.03787573j,  0.01341539+0.05313417j,
        -0.02138688+0.05045602j,  0.57905439+0.j        ]])

In [333]:
to_choi_from_super(example_process.full())

array([[ 0.57905442+0.j        ,  0.02138688-0.05045602j,
        -0.01341539-0.05313417j,  0.26332322+0.03787573j],
       [ 0.02138688+0.05045602j,  0.42094561+0.j        ,
         0.01191271-0.00902282j,  0.01341539+0.05313417j],
       [-0.01341539+0.05313417j,  0.01191271+0.00902282j,
         0.42094561+0.j        , -0.02138688+0.05045602j],
       [ 0.26332322-0.03787573j,  0.01341539-0.05313417j,
        -0.02138688-0.05045602j,  0.57905442+0.j        ]])

In [424]:
# def to_choi_from_super(superop):
#     J = np.zeros([2, 2, 2, 2], dtype=np.complex64)
#     for i in range(2):
#         for j in range(2):
#             for k in range(2):
#                 for l in range(2):
#                     ket = np.kron(qu.basis(2, i).full(), qu.basis(2, j).full())
#                     bra = np.kron(qu.basis(2, k).full(), qu.basis(2, l).full())
#                     J[i, j, k, l] = np.conjugate(bra.T) @ superop @ ket
# 
#     J_ijij = np.transpose(J, axes=[0, 2, 1, 3])
#     choi = 0
#     for i in range(2):
#         for j in range(2):
#             for k in range(2):
#                 for l in range(2):
#                     ket = np.kron(qu.basis(2, i).full(), qu.basis(2, j).full())
#                     bra = np.kron(qu.basis(2, k).full(), qu.basis(2, l).full())
#                     choi = choi + bra @ ket.T * J_ijij[i, j, k, l]
# 
#     return choi


def to_choi_from_super(superop):
    J = np.zeros([2, 2, 2, 2], dtype=np.complex64)
    basis = np.expand_dims(np.identity(2), axis=2)
    #column is basis[i]
    sum = 0
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):
                    sum = sum + np.kron(basis[l].T, basis[j].T) @ superop @ np.kron(basis[k], basis[i]) * np.kron(
                        basis[i], basis[j]) @ np.kron(basis[k].T, basis[l].T)

    return sum


def to_super_from_choi(choi):
    return to_choi_from_super(choi)


def to_chi_from_choi(choi):  # I think this is wrong. we need to correct it
    omega_ket = np.array(
        [np.kron(qu.basis(2, i), qu.basis(2, i)) for i in range(2)]
    ) / np.sqrt(2)

    chimn = np.zeros([4, 4], dtype=np.complex64)

    for i in range(4):
        for j in range(4):
            suma = 0
            for k in range(2):
                for l in range(2):
                    suma = suma + (
                            np.conjugate(omega_ket[l].T)
                            @ np.conjugate(np.kron(
                        np.identity(2), G[i],
                    ).T)
                            @ choi
                            @ np.kron(np.identity(2), G[j])
                            @ omega_ket[k]
                    )
            chimn[i, j] = suma

    return 2 * chimn


def to_super_from_chi(xi):
    # return np.einsum("ij,ijkl -> kl ",  xi, G_ijkl)
    A = np.zeros([4, 4], dtype=np.complex64)
    for i in range(4):
        for j in range(4):
            A = A + xi[i][j] * np.kron(np.transpose(G[j]), G[i])
    return A


def to_chi_from_super(superopmat):  #this is correct.
    aux = np.zeros([4, 4], dtype=np.complex64)

    for alpha in range(4):
        for beta in range(4):
            suma = 0
            for i in range(2):
                for j in range(2):
                    for k in range(2):
                        for l in range(2):
                            value = np.trace(G[beta].T @ (qu.basis(2, i) * qu.basis(2, k).dag()).full()) * np.trace(
                                G[alpha] @ (qu.basis(2, j) * qu.basis(2, l).dag()).full()) * np.kron(
                                qu.basis(2, i).dag().full(), qu.basis(2, j).dag().full()) @ superopmat @ np.kron(
                                qu.basis(2, k).full(), qu.basis(2, l).full())
                            suma = suma + value
            aux[alpha, beta] = suma
    return aux

In [265]:
def testAB(A, B):
    return np.isclose(A, B)

In [266]:
testAB(qu.to_choi(example_process), to_choi_from_super(example_process.full()))

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

In [122]:
testAB(qu.to_chi(example_process) / 4, to_chi_from_choi(qu.to_choi(example_process).full()))

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [123]:
testAB(to_super_from_chi((qu.to_chi(example_process) / 4).full()), example_process.full())

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

In [124]:
to_super_from_chi(qu.to_chi(example_process).full() / 2)

array([[ 0.57905439+0.j        , -0.01341539-0.05313417j,
        -0.01341539+0.05313417j,  0.42094561+0.j        ],
       [ 0.02138688-0.05045602j,  0.26332321+0.03787573j,
         0.01191271+0.00902282j, -0.02138688+0.05045602j],
       [ 0.02138688+0.05045602j,  0.01191271-0.00902282j,
         0.26332321-0.03787573j, -0.02138688-0.05045602j],
       [ 0.42094561+0.j        ,  0.01341539+0.05313417j,
         0.01341539-0.05313417j,  0.57905439+0.j        ]])

In [125]:
example_process.full()

array([[ 0.57905439+0.j        , -0.01341539+0.05313417j,
        -0.01341539-0.05313417j,  0.42094561+0.j        ],
       [ 0.02138688+0.05045602j,  0.26332321-0.03787573j,
         0.01191271-0.00902282j, -0.02138688-0.05045602j],
       [ 0.02138688-0.05045602j,  0.01191271+0.00902282j,
         0.26332321+0.03787573j, -0.02138688+0.05045602j],
       [ 0.42094561+0.j        ,  0.01341539-0.05313417j,
         0.01341539+0.05313417j,  0.57905439+0.j        ]])

In [129]:
qu.to_super(qu.to_chi(example_process))

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 0.57905439+0.j         -0.01341539+0.05313417j -0.01341539-0.05313417j
   0.42094561+0.j        ]
 [ 0.02138688+0.05045602j  0.26332321-0.03787573j  0.01191271-0.00902282j
  -0.02138688-0.05045602j]
 [ 0.02138688-0.05045602j  0.01191271+0.00902282j  0.26332321+0.03787573j
  -0.02138688+0.05045602j]
 [ 0.42094561+0.j          0.01341539-0.05313417j  0.01341539+0.05313417j
   0.57905439+0.j        ]]

In [190]:
example_chi = qu.rand_herm(4, seed=seed + 1).full()
example_chi

array([[ 0.        +0.j        , -0.55370597-0.01100925j,
        -0.15068285+0.71940151j,  0.        +0.j        ],
       [-0.55370597+0.01100925j, -0.33933036+0.j        ,
        -0.52852409-0.16862298j,  0.13284909-0.25329537j],
       [-0.15068285-0.71940151j, -0.52852409+0.16862298j,
         0.24226767+0.j        , -0.28641804-0.3608418j ],
       [ 0.        +0.j        ,  0.13284909+0.25329537j,
        -0.28641804+0.3608418j ,  0.5706703 +0.j        ]])

In [207]:
sum = 0
ket_ij = np.identity(4)
for i in range(4):
    for j in range(4):
        sum = sum + example_chi[i, j] * np.kron(
            G[j].T, G[i]
        )



In [208]:
sum

array([[ 0.28533515+0.j        , -0.7505501 +0.09740739j,
        -0.7505501 -0.09740739j,  0.12009164+0.j        ],
       [ 0.32969322-0.3396935j , -0.28533515+0.j        ,
        -0.29079901-0.52852409j, -0.52255738-0.06428472j],
       [ 0.32969322+0.3396935j , -0.29079901+0.52852409j,
        -0.28533515+0.j        , -0.52255738+0.06428472j],
       [-0.21715433+0.j        , -0.16399767-0.20001991j,
        -0.16399767+0.20001991j,  0.28533515+0.j        ]])

In [238]:
# superopmat = example_process.full()
superopmat = sum

In [242]:
aux

array([[ 0.        +0.j        , -0.553706  -0.01100925j,
        -0.15068285+0.71940154j,  0.        +0.j        ],
       [-0.553706  +0.01100925j, -0.33933035+0.j        ,
        -0.5285241 -0.16862299j,  0.13284908-0.25329536j],
       [-0.15068285-0.71940154j, -0.5285241 +0.16862299j,
         0.24226767+0.j        , -0.28641805-0.3608418j ],
       [ 0.        +0.j        ,  0.13284908+0.25329536j,
        -0.28641805+0.3608418j ,  0.5706703 +0.j        ]],
      dtype=complex64)

In [243]:
example_chi

array([[ 0.        +0.j        , -0.55370597-0.01100925j,
        -0.15068285+0.71940151j,  0.        +0.j        ],
       [-0.55370597+0.01100925j, -0.33933036+0.j        ,
        -0.52852409-0.16862298j,  0.13284909-0.25329537j],
       [-0.15068285-0.71940151j, -0.52852409+0.16862298j,
         0.24226767+0.j        , -0.28641804-0.3608418j ],
       [ 0.        +0.j        ,  0.13284909+0.25329537j,
        -0.28641804+0.3608418j ,  0.5706703 +0.j        ]])

In [237]:
(qu.to_chi(example_process) / 2).full()

array([[0.8423776 +0.j        , 0.        -0.10359019j,
        0.        -0.03480227j, 0.        -0.03787573j],
       [0.        +0.10359019j, 0.43285832+0.j        ,
        0.00902282+0.j        , 0.00797149+0.j        ],
       [0.        +0.03480227j, 0.00902282+0.j        ,
        0.40903289+0.j        , 0.00267815+0.j        ],
       [0.        +0.03787573j, 0.00797149+0.j        ,
        0.00267815+0.j        , 0.31573119+0.j        ]])

In [224]:
G

array([[[ 0.70710678+0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.70710678+0.j        ]],

       [[ 0.        +0.j        ,  0.70710678+0.j        ],
        [ 0.70710678+0.j        ,  0.        +0.j        ]],

       [[ 0.        +0.j        ,  0.        -0.70710678j],
        [ 0.        +0.70710678j,  0.        +0.j        ]],

       [[ 0.70710678+0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        , -0.70710678+0.j        ]]])

In [274]:
to_chi_from_super(to_super_from_chi(to_chi_from_choi(to_choi_from_super(to_super_from_chi(example_chi)))))

array([[ 0.        +0.j        , -0.55370593-0.01100925j,
        -0.15068285+0.71940154j,  0.        +0.j        ],
       [-0.55370593+0.01100925j, -0.33933038+0.j        ,
        -0.5285241 -0.16862297j,  0.1328491 -0.25329536j],
       [-0.15068285-0.71940154j, -0.5285241 +0.16862297j,
         0.24226767+0.j        , -0.28641805-0.3608418j ],
       [ 0.        +0.j        ,  0.1328491 +0.25329536j,
        -0.28641805+0.3608418j ,  0.5706703 +0.j        ]],
      dtype=complex64)

In [273]:
example_chi

array([[ 0.        +0.j        , -0.55370597-0.01100925j,
        -0.15068285+0.71940151j,  0.        +0.j        ],
       [-0.55370597+0.01100925j, -0.33933036+0.j        ,
        -0.52852409-0.16862298j,  0.13284909-0.25329537j],
       [-0.15068285-0.71940151j, -0.52852409+0.16862298j,
         0.24226767+0.j        , -0.28641804-0.3608418j ],
       [ 0.        +0.j        ,  0.13284909+0.25329537j,
        -0.28641804+0.3608418j ,  0.5706703 +0.j        ]])

In [271]:
testAB(to_chi_from_choi(to_choi_from_super(to_super_from_chi(example_chi)))
       , example_chi)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [276]:
to_chi_from_super(to_super_from_chi(example_chi))

array([[ 0.        +0.j        , -0.553706  -0.01100925j,
        -0.15068285+0.71940154j,  0.        +0.j        ],
       [-0.553706  +0.01100925j, -0.33933035+0.j        ,
        -0.5285241 -0.16862299j,  0.13284908-0.25329536j],
       [-0.15068285-0.71940154j, -0.5285241 +0.16862299j,
         0.24226767+0.j        , -0.28641805-0.3608418j ],
       [ 0.        +0.j        ,  0.13284908+0.25329536j,
        -0.28641805+0.3608418j ,  0.5706703 +0.j        ]],
      dtype=complex64)

In [277]:
to_chi_from_choi(to_choi_from_super(to_super_from_chi(example_chi)))

array([[ 0.        +0.j        , -0.55370593-0.01100925j,
        -0.15068285+0.71940154j,  0.        +0.j        ],
       [-0.55370593+0.01100925j, -0.33933038+0.j        ,
        -0.5285241 -0.16862297j,  0.1328491 -0.25329536j],
       [-0.15068285-0.71940154j, -0.5285241 +0.16862297j,
         0.24226767+0.j        , -0.28641805-0.3608418j ],
       [ 0.        +0.j        ,  0.1328491 +0.25329536j,
        -0.28641805+0.3608418j ,  0.5706703 +0.j        ]],
      dtype=complex64)

In [279]:
testAB(to_chi_from_choi(to_choi_from_super(to_super_from_chi(example_chi))),
       to_chi_from_super(to_super_from_chi(example_chi)))

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [283]:
to_chi_from_choi(qu.to_choi(example_process).full())

array([[ 4.2118880e-01-5.2041704e-18j,  2.7755576e-17-5.1795095e-02j,
         1.3530843e-16+1.7401133e-02j,  5.5511151e-17-1.8937865e-02j],
       [ 7.8062556e-18+5.1795095e-02j,  2.1642916e-01-3.9031278e-18j,
        -4.5114080e-03+4.1633363e-17j,  3.9857430e-03+1.3530843e-16j],
       [ 9.3675068e-17-1.7401133e-02j, -4.5114080e-03-4.1633363e-17j,
         2.0451644e-01+3.9031278e-18j, -1.3390746e-03-2.7755576e-17j],
       [-5.5511151e-17+1.8937865e-02j,  3.9857430e-03-9.3675068e-17j,
        -1.3390746e-03+7.8062556e-18j,  1.5786560e-01+5.2041704e-18j]],
      dtype=complex64)

In [285]:
to_chi_from_super(example_process.full()) / 2

array([[ 4.2118880e-01-5.2041704e-18j,  2.9923980e-17-5.1795095e-02j,
         1.3357371e-16+1.7401133e-02j,  2.7755576e-17-1.8937865e-02j],
       [ 7.3725748e-18+5.1795095e-02j,  2.1642916e-01-3.9031278e-18j,
        -4.5114080e-03+5.5511151e-17j,  3.9857430e-03+1.3357371e-16j],
       [ 9.7144515e-17-1.7401133e-02j, -4.5114080e-03-5.5511151e-17j,
         2.0451644e-01+3.9031278e-18j, -1.3390746e-03-2.8622937e-17j],
       [-2.7755576e-17+1.8937865e-02j,  3.9857430e-03-9.7144515e-17j,
        -1.3390746e-03+7.3725748e-18j,  1.5786560e-01+5.2041704e-18j]],
      dtype=complex64)

In [288]:
qu.to_chi(example_process) / 4

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True, superrep = chi
Qobj data =
[[0.4211888 +0.j         0.        -0.0517951j  0.        -0.01740113j
  0.        -0.01893787j]
 [0.        +0.0517951j  0.21642916+0.j         0.00451141+0.j
  0.00398574+0.j        ]
 [0.        +0.01740113j 0.00451141+0.j         0.20451645+0.j
  0.00133907+0.j        ]
 [0.        +0.01893787j 0.00398574+0.j         0.00133907+0.j
  0.15786559+0.j        ]]

In [420]:
basis = np.expand_dims(np.identity(2), axis=2)

np.kron(basis[1], basis[1]) @ np.kron(basis[0].T, basis[0].T)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [403]:
basis = np.identity(2)

np.kron(basis[1], basis[1]) @ np.kron(basis[1].T, basis[1].T)


1.0

In [425]:
to_choi_from_super(example_process.full())

array([[ 0.57905439+0.j        ,  0.02138688-0.05045602j,
        -0.01341539-0.05313417j,  0.26332321+0.03787573j],
       [ 0.02138688+0.05045602j,  0.42094561+0.j        ,
         0.01191271-0.00902282j,  0.01341539+0.05313417j],
       [-0.01341539+0.05313417j,  0.01191271+0.00902282j,
         0.42094561+0.j        , -0.02138688+0.05045602j],
       [ 0.26332321-0.03787573j,  0.01341539-0.05313417j,
        -0.02138688-0.05045602j,  0.57905439+0.j        ]])

In [428]:
testAB(
    qu.to_choi(example_process).full(),
    to_choi_from_super(example_process.full())
)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [426]:
qu.to_choi(example_process).full()

array([[ 0.57905439+0.j        ,  0.02138688-0.05045602j,
        -0.01341539-0.05313417j,  0.26332321+0.03787573j],
       [ 0.02138688+0.05045602j,  0.42094561+0.j        ,
         0.01191271-0.00902282j,  0.01341539+0.05313417j],
       [-0.01341539+0.05313417j,  0.01191271+0.00902282j,
         0.42094561+0.j        , -0.02138688+0.05045602j],
       [ 0.26332321-0.03787573j,  0.01341539-0.05313417j,
        -0.02138688-0.05045602j,  0.57905439+0.j        ]])

In [323]:
example_process

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 0.57905439+0.j         -0.01341539+0.05313417j -0.01341539-0.05313417j
   0.42094561+0.j        ]
 [ 0.02138688+0.05045602j  0.26332321-0.03787573j  0.01191271-0.00902282j
  -0.02138688-0.05045602j]
 [ 0.02138688-0.05045602j  0.01191271+0.00902282j  0.26332321+0.03787573j
  -0.02138688+0.05045602j]
 [ 0.42094561+0.j          0.01341539-0.05313417j  0.01341539+0.05313417j
   0.57905439+0.j        ]]

In [330]:
to_chi_from_choi(to_choi_from_super(to_super_from_chi(example_chi)))

array([[ 0.        +0.j        , -0.55370593-0.01100925j,
        -0.15068285+0.71940154j,  0.        +0.j        ],
       [-0.55370593+0.01100925j, -0.33933038+0.j        ,
        -0.5285241 -0.16862297j,  0.1328491 -0.25329536j],
       [-0.15068285-0.71940154j, -0.5285241 +0.16862297j,
         0.24226767+0.j        , -0.28641805-0.3608418j ],
       [ 0.        +0.j        ,  0.1328491 +0.25329536j,
        -0.28641805+0.3608418j ,  0.5706703 +0.j        ]],
      dtype=complex64)

In [331]:
example_chi

array([[ 0.        +0.j        , -0.55370597-0.01100925j,
        -0.15068285+0.71940151j,  0.        +0.j        ],
       [-0.55370597+0.01100925j, -0.33933036+0.j        ,
        -0.52852409-0.16862298j,  0.13284909-0.25329537j],
       [-0.15068285-0.71940151j, -0.52852409+0.16862298j,
         0.24226767+0.j        , -0.28641804-0.3608418j ],
       [ 0.        +0.j        ,  0.13284909+0.25329537j,
        -0.28641804+0.3608418j ,  0.5706703 +0.j        ]])